In [48]:
import pandas as pd
import numpy as np
import pycountry as pc
import folium
from sklearn import preprocessing
from IPython.display import HTML

We start with the analysis of the first DataSet which is about the crops. We are taking into account only the years after 2000, for the moment. 

In [49]:
#loading the DataSet
path = 'data/Production_Crops_E_All_Data_NOFLAG.csv'
crops = pd.read_csv(path, encoding = "ISO-8859-1")

In [50]:
crops.head()

Area Code         Area  Item Code                              Item  \
0          2  Afghanistan        221               Almonds, with shell   
1          2  Afghanistan        221               Almonds, with shell   
2          2  Afghanistan        221               Almonds, with shell   
3          2  Afghanistan        711  Anise, badian, fennel, coriander   
4          2  Afghanistan        711  Anise, badian, fennel, coriander   

   Element Code         Element    Unit  Y1961  Y1962  Y1963  ...    Y2008  \
0          5312  Area harvested      ha    NaN    NaN    NaN  ...  12000.0   
1          5419           Yield   hg/ha    NaN    NaN    NaN  ...  35000.0   
2          5510      Production  tonnes    NaN    NaN    NaN  ...  42000.0   
3          5312  Area harvested      ha    NaN    NaN    NaN  ...  15000.0   
4          5419           Yield   hg/ha    NaN    NaN    NaN  ...   6000.0   

     Y2009    Y2010    Y2011    Y2012    Y2013    Y2014    Y2015    Y2016  \
0  11029.0  11210.0  13469.0  13490.0  14114.0  13703.0  14676.0  19481.0   
1  39154.0  49955.0  45000.0  45960.0  29910.0  19996.0  16521.0  16859.0   
2  43183.0  56000.0  60611.0  62000.0  42215.0  27400.0  24246.0  32843.0   
3  17748.0  17000.0  19500.0  18500.0  18500.0  30000.0  25000.0  26019.0   
4   6203.0   6000.0   6414.0   6757.0   6757.0   7167.0   7200.0   6923.0   

     Y2017  
0  19793.0  
1  13788.0  
2  27291.0  
3  28873.0  
4   6830.0  

[5 rows x 64 columns]

As we can see for the DS for each item, for each country there are three options fro the unit parameter. 
<br>
Production Quantity and Seed: tonnes
<br>
Area harvested: hectares
<br>
Yield: tonnes per hectare.

We decided to keep only Production, this beacuse this feature is the more significant for the purpose of the project in general. **we are probably going to consider the *Area harvest* when we will talk about the energy source consuption**

In [51]:
crops_production = crops[crops['Element']=='Production']
crops_yield = crops[crops['Element']=='Yield']

In [52]:
#years we need to drop
years = ['Y'+str(1900+x) for x in range(61,100)]
crops_production = crops_production.drop(years, axis=1)

In [53]:
crops_production.head()

Area Code         Area  Item Code                              Item  \
2           2  Afghanistan        221               Almonds, with shell   
5           2  Afghanistan        711  Anise, badian, fennel, coriander   
8           2  Afghanistan        515                            Apples   
11          2  Afghanistan        526                          Apricots   
14          2  Afghanistan         44                            Barley   

    Element Code     Element    Unit    Y2000    Y2001     Y2002  ...  \
2           5510  Production  tonnes  12000.0  15000.0   11774.0  ...   
5           5510  Production  tonnes   2500.0   1000.0    2000.0  ...   
8           5510  Production  tonnes  40540.0  39542.0   35630.0  ...   
11          5510  Production  tonnes  42840.0  38000.0   45758.0  ...   
14          5510  Production  tonnes  74000.0  87000.0  345000.0  ...   

       Y2008     Y2009     Y2010     Y2011     Y2012     Y2013     Y2014  \
2    42000.0   43183.0   56000.0   60611.0   62000.0   42215.0   27400.0   
5     9000.0   11008.0   10200.0   12508.0   12500.0   12500.0   21500.0   
8    56061.0   72765.0   59850.0   62041.0   70000.0   78597.0   89403.0   
11   59559.0   60000.0   66560.0   67995.0   83500.0   90000.0   90000.0   
14  333000.0  486000.0  437000.0  305900.0  504000.0  514000.0  521000.0   

       Y2015     Y2016     Y2017  
2    24246.0   32843.0   27291.0  
5    18000.0   18012.0   19719.0  
8    89733.0  140903.0  175000.0  
11   87686.0   17894.0  131816.0  
14  403000.0  301856.0   94995.0  

[5 rows x 25 columns]

In [54]:
#crops_production['Area'] = crops_production['Area'].replace('United States of America', 'United States')
changes = {'Area' : { 'United States of America' : 'United States' ,
                    'United Republic of Tanzania' : 'Tanzania, United Republic of',
                    "Democratic People's Republic of Korea" : "Korea, Democratic People's Republic of",
                    "Republic of Korea" : 'SKorea, Republic of',
                    "The former Yugoslav Republic of Macedonia" : 'North Macedonia',
                    "Democratic Republic of the Congo" : 'Congo, The Democratic Republic of the',
                    "Bolivia (Plurinational State of)" : 'Bolivia, Plurinational State of',
                    "Venezuela (Bolivarian Republic of)" : 'Venezuela, Bolivarian Republic of',
                    "Republic of Moldova" : 'Moldova, Republic of',
                    # "Republic of Moldova" : 'Moldova, Republic of',
                    }}
crops_production= crops_production.replace(changes)

We are now intereste in what are the possible **Item** producted.

In [55]:
#crops_production['Item'].unique()[:10]

For the moment we decided not to clean the Item column. Indeed we are considering the element producted in term of tonnes.

Now we need to classify the country in terms of production, wed like to remember we are taking into account only the last years.

In [56]:
crops_production = crops_production.groupby('Area').sum()
to_drop = ['Area Code','Item Code','Element Code']
crops_production = crops_production.drop(to_drop, axis=1)

We have to drop the index which are not rapresenting a country, but a region

In [57]:
countries = [i.name for i in pc.countries]
to_delete = set(crops_production.index).difference(countries)
crops_production = crops_production.drop(to_delete, axis=0)

In [58]:
def norm(col):
    return col/sum(col)

In [59]:
crops_production['tot'] = crops_production.apply(norm).mean(axis=1)

In [60]:
result = crops_production.sort_values(by=['tot'], ascending=0)
result = result[['tot']]

In [62]:
code = [pc.countries.get(name=i).alpha_3 for i in result.index]
result['code'] = code
result['Area'] = result.index
result['tot'] = result['tot']*100
result = result[result['tot']>=1]
result.head(10)

tot code                Area
Area                                                  
China               21.360634  CHN               China
United States       11.923088  USA       United States
India               10.103422  IND               India
Brazil               7.184247  BRA              Brazil
Indonesia            3.141164  IDN           Indonesia
Russian Federation   2.697834  RUS  Russian Federation
Nigeria              2.121509  NGA             Nigeria
France               1.974328  FRA              France
Argentina            1.828803  ARG           Argentina
Thailand             1.701358  THA            Thailand

In [65]:
country_geo = 'world-countries.json'
# Setup a folium map at a high-level zoom
m = folium.Map(location=[0, 0], zoom_start=1.5)


folium.Choropleth(
    geo_data=country_geo,
    name='choropleth',
    data=result,
    columns=['code', 'tot'],
    key_on='feature.id',
    fill_color='YlGn',
    threshold_scale=[0, 1, 2, 4, 20, 32]
).add_to(m)
m.save("crops_production.html")
m

In [75]:
def mapp(df):
    country_geo = 'world-countries.json'
    # Setup a folium map at a high-level zoom
    m = folium.Map(location=[0, 0], zoom_start=1.5)


    folium.Choropleth(
        geo_data=country_geo,
        name='choropleth',
        data=result,
        columns=['code', 'tot'],
        key_on='feature.id',
        fill_color='YlGn',
        threshold_scale=[0, 1, 2, 4, 20, 32]
    ).add_to(m)
    m.save(df+".html")
    return m

In [18]:
def clean_df(df,years,to_drop,changes):
    df= df.replace(changes)
    df = df.drop(years, axis=1)
    df = df.groupby('Area').sum()
    df = df.drop(to_drop, axis=1)
    to_delete = set(df.index).difference(countries)
    df = df.drop(to_delete, axis=0)
    df['tot'] = 0
    df['tot'] = df.apply(norm).mean(axis=1)
    df = df.sort_values(by=['tot'], ascending=0)
    df = df[['tot']]
    return df

We reapeat the some for the DS Yield

In [19]:
to_drop = ['Area Code','Item Code','Element Code']
result = clean_df(crops_yield,years,to_drop,changes)

In [20]:
code = [pc.countries.get(name=i).alpha_3 for i in result.index]
result['code'] = code
result['Area'] = result.index
result['tot'] = result['tot']*100
result = result[result['tot']>=1]
result.head(10)

tot code            Area
Area                                         
Netherlands     5.659627  NLD     Netherlands
Belgium         2.050386  BEL         Belgium
Spain           1.669163  ESP           Spain
United Kingdom  1.563273  GBR  United Kingdom
United States   1.493866  USA   United States
China           1.464058  CHN           China
Greece          1.340356  GRC          Greece
Israel          1.284369  ISR          Israel
France          1.236503  FRA          France
Germany         1.230535  DEU         Germany

In [77]:
m = mapp('crops_yield')
m

In [22]:
path = 'data/Production_CropsProcessed_E_All_Data.csv'
crops_processed = pd.read_csv(path, encoding = "ISO-8859-1")

In [23]:
crops_processed.head()

Area Code         Area  Item Code             Item  Element Code  \
0          2  Afghanistan        767      Cotton lint          5510   
1          2  Afghanistan        329       Cottonseed          5510   
2          2  Afghanistan        165         Molasses          5510   
3          2  Afghanistan        331  Oil, cottonseed          5510   
4          2  Afghanistan        334     Oil, linseed          5510   

      Element    Unit    Y1961 Y1961F    Y1962  ...    Y2010  Y2010F    Y2011  \
0  Production  tonnes  17000.0      *  26000.0  ...  10890.0      Im  10890.0   
1  Production  tonnes  34000.0      *  52000.0  ...  21450.0      Im  21450.0   
2  Production  tonnes   2120.0     Fc   3023.0  ...   1210.0      Im   1212.0   
3  Production  tonnes   4997.0     Fc   7716.0  ...   3653.0      Im   3653.0   
4  Production  tonnes   3531.0     Fc   3701.0  ...    356.0      Im    784.0   

   Y2011F    Y2012  Y2012F    Y2013  Y2013F    Y2014  Y2014F  
0      Im  11979.0      Im  13917.0      Im  14000.0       F  
1      Im  23595.0      Im  42173.0     NaN  42829.0     NaN  
2      Im   1153.0      Im   1188.0      Im   1017.0      Im  
3      Im   4018.0      Im   4714.0      Im   7489.0      Im  
4      Im    784.0      Im    784.0      Im    790.0       F  

[5 rows x 115 columns]

In [24]:
years = ['Y'+str(1900+x) for x in range(61,100)] + ['Y'+str(1900+x)+'F' for x in range(61,100)] +['Y'+str(2000+x)+'F' for x in range(0,15)]
to_drop = ['Area Code','Item Code','Element Code']
#code = [pc.countries.get(name=i).alpha_3 for i in result.index]

In [25]:
result = clean_df(crops_processed,years,to_drop,changes)

In [26]:
code = [pc.countries.get(name=i).alpha_3 for i in result.index]
result['code'] = code
result['Area'] = result.index
result['tot'] = result['tot']*100
result = result[result['tot']>=1]
result.head(10)

tot code                Area
Area                                                  
China               14.346097  CHN               China
Brazil              10.601861  BRA              Brazil
United States        9.953984  USA       United States
India                8.390099  IND               India
Indonesia            4.507170  IDN           Indonesia
Malaysia             3.631136  MYS            Malaysia
Germany              3.157906  DEU             Germany
Mexico               2.686162  MEX              Mexico
Russian Federation   2.627587  RUS  Russian Federation
Pakistan             2.320982  PAK            Pakistan

In [78]:
m = mapp('crops_processed')
m


In [28]:
path = 'data/Production_LivestockPrimary_E_All_Data_NOFLAG.csv'
live_stock = pd.read_csv(path, encoding = "ISO-8859-1")

In [29]:
live_stock.head()

Area Code         Area  Item Code                          Item  \
0          2  Afghanistan       1062           Eggs, hen, in shell   
1          2  Afghanistan       1062           Eggs, hen, in shell   
2          2  Afghanistan       1062           Eggs, hen, in shell   
3          2  Afghanistan       1067  Eggs, hen, in shell (number)   
4          2  Afghanistan        919          Hides, cattle, fresh   

   Element Code                        Element       Unit     Y1961     Y1962  \
0          5313                         Laying  1000 Head    4000.0    4400.0   
1          5410                          Yield   100mg/An   25000.0   25000.0   
2          5510                     Production     tonnes   10000.0   11000.0   
3          5513                     Production    1000 No  200000.0  220000.0   
4          5320  Producing Animals/Slaughtered       Head  360000.0  384000.0   

      Y1963  ...     Y2008     Y2009     Y2010     Y2011     Y2012     Y2013  \
0    4600.0  ...    9100.0    8700.0    8200.0    9600.0    9500.0    9500.0   
1   25000.0  ...   18349.0   19285.0   19932.0   17927.0   18947.0   18947.0   
2   11500.0  ...   16698.0   16778.0   16344.0   17210.0   18000.0   18000.0   
3  230000.0  ...  333960.0  335560.0  326880.0  344200.0  360000.0  360000.0   
4  396000.0  ...  739000.0  750000.0  727800.0  766700.0  772000.0  744500.0   

      Y2014     Y2015     Y2016     Y2017  
0    9500.0    9337.0    9369.0   10688.0  
1   19474.0   21253.0   21263.0   18713.0  
2   18500.0   19844.0   19921.0   20000.0  
3  370000.0  396880.0  398420.0  400000.0  
4       NaN       NaN       NaN       NaN  

[5 rows x 64 columns]

In [30]:
live_stock_production = live_stock[live_stock['Element']=='Production']
live_stock_yield = live_stock[live_stock['Element']=='Yield']

In [31]:
years = ['Y'+str(1900+x) for x in range(61,100)] #+ ['Y'+str(1900+x)+'F' for x in range(61,100)] +['Y'+str(2000+x)+'F' for x in range(0,15)]
to_drop = ['Area Code','Item Code','Element Code']

In [32]:
result = clean_df(live_stock_production,years,to_drop,changes)

In [33]:
code = [pc.countries.get(name=i).alpha_3 for i in result.index]
result['code'] = code
result['Area'] = result.index
result['tot'] = result['tot']*100
result = result[result['tot']>=1]
result.head(10)

tot code                Area
Area                                                  
China               27.438780  CHN               China
United States       10.031018  USA       United States
India                7.249769  IND               India
Brazil               4.714211  BRA              Brazil
Russian Federation   2.941969  RUS  Russian Federation
Germany              2.457233  DEU             Germany
France               2.131816  FRA              France
Pakistan             2.120995  PAK            Pakistan
Mexico               2.119883  MEX              Mexico
Spain                1.649480  ESP               Spain

In [79]:
m = mapp('live_stock_production')
m

In [37]:
path = 'data/Value_of_Production_E_All_Data_NOFLAG.csv'
Value_of_production = pd.read_csv(path, encoding = "ISO-8859-1")

In [38]:
Value_of_production.head()

Area Code         Area  Item Code                              Item  \
0          2  Afghanistan        221               Almonds, with shell   
1          2  Afghanistan        221               Almonds, with shell   
2          2  Afghanistan        221               Almonds, with shell   
3          2  Afghanistan        221               Almonds, with shell   
4          2  Afghanistan        711  Anise, badian, fennel, coriander   

   Element Code                                            Element  \
0           152  Gross Production Value (constant 2004-2006 100...   
1           154  Net Production Value (constant 2004-2006 1000 I$)   
2            56       Gross Production Value (current million SLC)   
3            55  Gross Production Value (constant 2004-2006 mil...   
4           152  Gross Production Value (constant 2004-2006 100...   

          Unit  Y1961  Y1962  Y1963  ...         Y2007          Y2008  \
0  1000 Int. $    NaN    NaN    NaN  ...  92899.455089  123940.698000   
1  1000 Int. $    NaN    NaN    NaN  ...  92899.455089  123940.698000   
2          SLC    NaN    NaN    NaN  ...   5433.830473    8980.020000   
3          SLC    NaN    NaN    NaN  ...   3280.355179    4376.446667   
4  1000 Int. $    NaN    NaN    NaN  ...  93962.672000   49744.944000   

           Y2009          Y2010          Y2011          Y2012          Y2013  \
0  127431.694327  165254.264000  178861.182059  182960.078000  124575.156335   
1  127431.694327  165254.264000  178861.182059  182960.078000  124575.156335   
2    8032.038000   12079.200000   13995.079900   14911.000000    9230.459191   
3    4499.716581    5835.262222    6315.733546    6460.468889    4398.849906   
4   57275.009778   56377.603200   64207.463034   69090.200000   69090.200000   

          Y2014         Y2015         Y2016  
0  80856.550600  71549.194374  96918.674867  
1  80856.550600  71549.194374  96918.674867  
2   5563.109132   4824.773367   6050.024466  
3   2855.110444   2526.460140   3422.277092  
4  73114.852434  79457.968291  77478.699955  

[5 rows x 63 columns]

In [39]:
Value_of_production_NET = Value_of_production[Value_of_production['Element']==
                                              'Net Production Value (constant 2004-2006 1000 I$)']

In [40]:
years = ['Y'+str(1900+x) for x in range(61,100)] #+ ['Y'+str(1900+x)+'F' for x in range(61,100)] +['Y'+str(2000+x)+'F' for x in range(0,15)]
to_drop = ['Area Code','Item Code','Element Code']

In [41]:
result = clean_df(Value_of_production_NET,years,to_drop,changes)

In [42]:
code = [pc.countries.get(name=i).alpha_3 for i in result.index]
result['code'] = code
result['Area'] = result.index
result['tot'] = result['tot']*100
result = result[result['tot']>=1]
result.head(10)

tot code                Area
Area                                                  
China               23.102057  CHN               China
United States       10.470934  USA       United States
India               10.081729  IND               India
Brazil               5.886076  BRA              Brazil
Indonesia            2.589076  IDN           Indonesia
Russian Federation   2.036014  RUS  Russian Federation
France               1.984159  FRA              France
Argentina            1.819351  ARG           Argentina
Pakistan             1.677693  PAK            Pakistan
Germany              1.669351  DEU             Germany

In [80]:
m = mapp('Value_of_production_NET')
m